In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
from solver_shifts import run_solver_shift_return


/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## Copied and Pasted from Alberto's readme


The solver is a single Python script, contained in `solver/solver.py`.
It takes the following parameters:
* `-m` to specify the model name (`base`, `fixed`, `partflex`, or `flex`).
* `-i` to specify the location of the instance file.
* `-c` to specify the outsourcing cost multiplier (parameter OC in the paper).
* `-r` to specify the regional bound multiplier (parameter RM in the paper).
* `-g` to specify the global bound multiplier (parameter GM in the paper).
* `-u` to specify the maximum number of shift start times for the `partflex` model (parameter $\mu$ in the paper).
* `-o` to specify the location of the JSON solution file produced by the solver.

In [17]:
#Code that creates JSON file that outputs optimal shifts from scheduling solution

# for file in ['lyon_db=0.50_dt=uniform.json']:
for file in os.listdir(r'../instances'):
    if file not in os.listdir(r'../shifts'):
        shift_out = f"../shifts/{file}"
        dict_out = {
            'instance_file':[],
            'city':[],
            'demand_baseline':[],
            'demand_type':[],
            'model':[],
            'max_n_shifts':[],
            'outsourcing_cost_multiplier':[],
            'regional_multiplier':[],
            'global_multiplier':[],
            'run_time':[],
            'obj_val': [], 
            'status': [], 
            'gap': [],
            'region':[],
            'shifts_start':[],
            'shifts_end':[]
        }

        city_pattern = r'(\w+)_db'
        db_pattern = r'db=(\d+\.\d+)'
        dt_pattern = r'dt=(\w+)'

        city_match = re.search(city_pattern, file)
        db_match = re.search(db_pattern, file)
        dt_match = re.search(dt_pattern, file)
        
        city = city_match.group(1) if city_match else None
        demand_baseline = float(db_match.group(1)) if db_match else None
        demand_type = dt_match.group(1) if dt_match else None

        instance_ = f"../instances/{file}"

        for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
            for RM in [0.75, 1.00, 1.50, 1.80, 2.00]:
                for GM in [0.6, 0.7, 0.8, 0.9, 1.0]:
                    for model in ['fixed','flex','partflex']:
                        if model == 'partflex':
                            for max_n_shift in range(2,5):
                                dict_shifts, n_regions, dict_raw = run_solver_shift_return(model=model, instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM, max_n_shifts=max_n_shift)
                                # if max_n_shift == 2:
                                #     dict_raw_out = dict_raw.copy()
                                dict_base = {
                                    'instance_file':[file]*n_regions,
                                    'city':[city]*n_regions,
                                    'demand_baseline':[demand_baseline]*n_regions,
                                    'demand_type':[demand_type]*n_regions,
                                    'model':[model]*n_regions,
                                    'max_n_shifts':[max_n_shift]*n_regions,
                                    'outsourcing_cost_multiplier':[OC]*n_regions,
                                    'regional_multiplier':[RM]*n_regions,
                                    'global_multiplier':[GM]*n_regions,
                                    'run_time': [dict_raw['run_time'][0]]*n_regions,
                                    'obj_val': [dict_raw['obj_val'][0]]*n_regions,
                                    'status': [dict_raw['status'][0]]*n_regions,
                                    'gap': [dict_raw['gap'][0]]*n_regions,
                                    'region':[region for region in range(0,n_regions)]
                                }
                                list_shift_start = []
                                list_shift_end = []
                                for region in range(0, n_regions):
                                    if region in dict_shifts.keys():
                                        list_shift_start.append(dict_shifts[region]['shifts_start'])
                                        list_shift_end.append(dict_shifts[region]['shifts_end'])
                                    else:
                                        list_shift_start.append({})
                                        list_shift_end.append({})
                                dict_base['shifts_start'] = list_shift_start
                                dict_base['shifts_end'] = list_shift_end
                                for key in dict_base.keys():
                                    dict_out[key].extend(dict_base[key])
                        else:
                            dict_shifts, n_regions, dict_raw = run_solver_shift_return(model=model, instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)
                            dict_base = {
                                'instance_file':[file]*n_regions,
                                'city':[city]*n_regions,
                                'demand_baseline':[demand_baseline]*n_regions,
                                'demand_type':[demand_type]*n_regions,
                                'model':[model]*n_regions,
                                'max_n_shifts':[np.nan]*n_regions,
                                'outsourcing_cost_multiplier':[OC]*n_regions,
                                'regional_multiplier':[RM]*n_regions,
                                'global_multiplier':[GM]*n_regions,
                                'run_time': [dict_raw['run_time'][0]]*n_regions,
                                'obj_val': [dict_raw['obj_val'][0]]*n_regions,
                                'status': [dict_raw['status'][0]]*n_regions,
                                'gap': [dict_raw['gap'][0]]*n_regions,
                                'region':[region for region in range(0,n_regions)]
                            }
                            list_shift_start = []
                            list_shift_end = []
                            for region in range(0, n_regions):
                                if region in dict_shifts.keys():
                                    list_shift_start.append(dict_shifts[region]['shifts_start'])
                                    list_shift_end.append(dict_shifts[region]['shifts_end'])
                                else:
                                    list_shift_start.append({})
                                    list_shift_end.append({})
                            dict_base['shifts_start'] = list_shift_start
                            dict_base['shifts_end'] = list_shift_end
                            for key in dict_base.keys():
                                dict_out[key].extend(dict_base[key])

        with open(shift_out, 'w') as f:
            json.dump(dict_out, f, indent=2)


in instance
in solver
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4000 rows, 4368 columns and 8356 nonzeros
Model fingerprint: 0xf674535d
Variable types: 3840 continuous, 528 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 515.8400007
Found heuristic solution: objective 515.8399964
Presolve removed 3856 rows and 3989 columns
Presolve time: 0.01s
Presolved: 144 rows, 379 columns, 849 nonzeros
Found heuristic solution: objective 437.0066667
Variable types: 165 continuous, 214 integer (0 binary)

Root relaxation: objective 3.571000e+02, 86 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curre

In [19]:
#this is an example output of the shift creation code

list_output = []

for filename in os.listdir(r'../shifts'):
    if filename.find('json')>-1:
        with open(f'../shifts/{filename}', 'r') as file:
            data = json.load(file)
            df_ = pd.DataFrame(data)
            list_output.append(df_)

df_output = pd.concat(list_output, ignore_index = True)
df_output.to_excel(r'../run_code/df_shifts_example.xlsx', index = False)


In [5]:
df_output.head()

,instance_file,city,demand_baseline,demand_type,model,max_n_shifts,outsourcing_cost_multiplier,regional_multiplier,global_multiplier,region,shifts_start,shifts_end
0,paris_db=2.00_dt=doublepeak.json,paris,2.0,doublepeak,fixed,NaN,1.2,1.5,0.8,0,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
1,paris_db=2.00_dt=doublepeak.json,paris,2.0,doublepeak,fixed,NaN,1.2,1.5,0.8,1,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
2,paris_db=2.00_dt=doublepeak.json,paris,2.0,doublepeak,fixed,NaN,1.2,1.5,0.8,2,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
3,paris_db=2.00_dt=doublepeak.json,paris,2.0,doublepeak,fixed,NaN,1.2,1.5,0.8,3,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
4,paris_db=2.00_dt=doublepeak.json,paris,2.0,doublepeak,flex,NaN,1.2,1.5,0.8,0,"{'0': 0, '1': 2, '2': 4}","{'0': 4, '1': 6, '2': 8}"


In [6]:
df_freq = df_output.copy()
df_freq.drop_duplicates(subset = ['shifts_start'], inplace = True)
df_freq

,instance_file,city,demand_baseline,demand_type,model,max_n_shifts,outsourcing_cost_multiplier,regional_multiplier,global_multiplier,region,shifts_start,shifts_end
0,paris_db=2.00_dt=doublepeak.json,paris,2.0,doublepeak,fixed,NaN,1.2,1.5,0.8,0,"{'0': 0, '1': 4}","{'0': 4, '1': 8}"
4,paris_db=2.00_dt=doublepeak.json,paris,2.0,doublepeak,flex,NaN,1.2,1.5,0.8,0,"{'0': 0, '1': 2, '2': 4}","{'0': 4, '1': 6, '2': 8}"
704,berlin_db=1.00_dt=uniform.json,berlin,1.0,uniform,flex,NaN,1.2,1.5,0.8,0,"{'0': 0, '1': 3, '2': 4}","{'0': 4, '1': 7, '2': 8}"
707,berlin_db=1.00_dt=uniform.json,berlin,1.0,uniform,flex,NaN,1.2,1.5,0.8,3,"{'0': 0, '1': 1, '2': 4}","{'0': 4, '1': 5, '2': 8}"
1204,berlin_db=0.50_dt=doublepeak.json,berlin,0.5,doublepeak,flex,NaN,1.2,1.5,0.8,0,{'0': 2},{'0': 6}
1305,frankfurt_db=0.50_dt=doublepeak.json,frankfurt,0.5,doublepeak,flex,NaN,1.2,1.5,0.8,1,"{'0': 2, '1': 3}","{'0': 6, '1': 7}"
1344,frankfurt_db=0.50_dt=doublepeak.json,frankfurt,0.5,doublepeak,flex,NaN,1.8,1.5,0.8,0,"{'0': 2, '1': 4}","{'0': 6, '1': 8}"
